In [6]:
import pickle
import numpy as np

In [7]:
task_name = "task1, task2, taskNRv2"

In [8]:
''' set up dataloader '''

whole_dataset_dicts = []

if 'task1' in task_name:
    dataset_path_task1 = r'I:\Science\CIS-YASHMOSH\niallmcguire\ZuCo\task1-SR\pickle\task1-SR-dataset.pickle'
    with open(dataset_path_task1, 'rb') as handle:
        whole_dataset_dicts.append(pickle.load(handle))

if 'task2' in task_name:
    dataset_path_task2 = r'I:\Science\CIS-YASHMOSH\niallmcguire\ZuCo\task2-NR\pickle\task2-NR-dataset.pickle'
    with open(dataset_path_task2, 'rb') as handle:
        whole_dataset_dicts.append(pickle.load(handle))

if 'task3' in task_name:
    dataset_path_task3 = r'I:\Science\CIS-YASHMOSH\niallmcguire\ZuCo\task3-TSR\pickle\task3-TSR-dataset.pickle'
    with open(dataset_path_task3, 'rb') as handle:
        whole_dataset_dicts.append(pickle.load(handle))

if 'taskNRv2' in task_name:
    dataset_path_taskNRv2 = r'I:\Science\CIS-YASHMOSH\niallmcguire\ZuCo\task2-NR-2.0\pickle\task2-NR-2.0-dataset.pickle'
    with open(dataset_path_taskNRv2, 'rb') as handle:
        whole_dataset_dicts.append(pickle.load(handle))

In [9]:
subject_choice = 'ALL'
eeg_type_choice = 'GD'
bands_choice = ['_t1','_t2','_a1','_a2','_b1','_b2','_g1','_g2']
dataset_setting = 'unique_sent'

In [10]:
Task_Dataset_List = whole_dataset_dicts
if not isinstance(whole_dataset_dicts,list):
    Task_Dataset_List = [whole_dataset_dicts]

In [11]:
def get_eeg_word_embedding(word, eeg_type = 'GD', bands = ['_t1','_t2','_a1','_a2','_b1','_b2','_g1','_g2']):
    EEG_frequency_features = []
    EEG_word_level_label = word['content']
    for band in bands:
        EEG_frequency_features.append(word['word_level_EEG'][eeg_type][eeg_type+band])
    word_eeg_embedding = np.concatenate(EEG_frequency_features)
    if len(word_eeg_embedding) != 105*len(bands):
        print(f'expect word eeg embedding dim to be {105*len(bands)}, but got {len(word_eeg_embedding)}, return None')
        word_eeg_embedding = None
    else:
        word_eeg_embedding = word_eeg_embedding.reshape(105, 8)

    return word_eeg_embedding, EEG_word_level_label





In [12]:
import torch
EEG_word_level_embeddings = []
EEG_word_level_labels = []
EEG_Sentences = []
#Main loop, looping through each task
for Task_Dataset in Task_Dataset_List:
    subjects = list(Task_Dataset.keys())
    print('[INFO]using subjects: ', subjects)

    total_num_sentence = len(Task_Dataset[subjects[0]])

    train_divider = int(0.8*total_num_sentence)
    dev_divider = train_divider + int(0.1*total_num_sentence)

    print(f'train size = {train_divider}')
    print(f'dev size = {dev_divider}')


    print('[INFO]initializing a train set...')
    for key in subjects:
        print(f'key = {key}')
        for i in range(train_divider):
            if Task_Dataset[key][i] is not None:
                sentence_object = Task_Dataset[key][i]
                sentence = sentence_object['content']

                #print(sentence_object['content'])
                none_switch = False

                for word in sentence_object['word']:
                    word_eeg_embedding, EEG_word_level_label = get_eeg_word_embedding(word)
                    if word_eeg_embedding is not None and torch.isnan(torch.from_numpy(word_eeg_embedding)).any() == False:
                        EEG_word_level_embeddings.append(word_eeg_embedding)
                        EEG_word_level_labels.append(EEG_word_level_label)
                    else:
                        none_switch = True
                if none_switch == False:
                    EEG_Sentences.append(sentence)


[INFO]using subjects:  ['ZAB', 'ZDM', 'ZDN', 'ZGW', 'ZJM', 'ZJN', 'ZJS', 'ZKB', 'ZKH', 'ZMG', 'ZPH']
train size = 320
dev size = 360
[INFO]initializing a train set...
key = ZAB
key = ZDM
key = ZDN
key = ZGW
key = ZJM
key = ZJN
key = ZJS
key = ZKB
key = ZKH
key = ZMG
key = ZPH
[INFO]using subjects:  ['ZAB', 'ZDM', 'ZDN', 'ZGW', 'ZJM', 'ZJN', 'ZJS', 'ZKB', 'ZKH', 'ZKW', 'ZMG', 'ZPH']
train size = 240
dev size = 270
[INFO]initializing a train set...
key = ZAB
key = ZDM
key = ZDN
key = ZGW
key = ZJM
key = ZJN
key = ZJS
key = ZKB
key = ZKH
key = ZKW
key = ZMG
key = ZPH
[INFO]using subjects:  ['YAC', 'YAG', 'YAK', 'YDG', 'YDR', 'YFR', 'YFS', 'YHS', 'YIS', 'YLS', 'YMD', 'YMS', 'YRH', 'YRK', 'YRP', 'YSD', 'YSL', 'YTL']
train size = 279
dev size = 313
[INFO]initializing a train set...
key = YAC
expect word eeg embedding dim to be 840, but got 0, return None
expect word eeg embedding dim to be 840, but got 0, return None
expect word eeg embedding dim to be 840, but got 0, return None
expect word

In [13]:
len(EEG_word_level_embeddings)

145247